# Symétries
### Louis Noirot, mars 2025
###### LMAPR1492

In [112]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
#from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [113]:
mp_key = "g2nCFD5rMkPRt9qdpOGhbfHJf2mgbv5x"
mp_id = "mp-14116"

In [114]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [115]:
# Visualisation
view = quick_view(structure)
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [116]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [117]:
# Symétries de la molécule
sym = SpacegroupAnalyzer(structure).get_symmetry_operations()

# Symétries choisies pour les atomes
# Plan miroir (z,y,x)
S1 = sym[7]
# Axe 4-barre | centre d'inversion en (0,0,0)  (-x-y-z,x,y)
S2 = sym[3]
# Axe de rotation C2 (y,x,-x-y-z)
S3 = sym[6]

# Atomes choisis (positions fractionnaires dans CuRhO2)
Cu = [0.0, 0.0, 0.0]
Rh = [0.5, 0.5, 0.5]
O = [0.25, 0.25, 0.75]

# Application des symétries
Cu_sym = S1.operate(Cu)
Rh_sym = S2.operate(Rh)
O_sym = S3.operate(O)

# Application des oppérations de symétrie
transformed_positions = [S1.operate(atom_positions[0]),
                         S2.operate(atom_positions[1]),
                         S3.operate(atom_positions[2])]

# Affichage des résultats
print("Position originale et transformée des atomes:")
print(f"Cu: {Cu} -> {Cu_sym}")
print(f"Rh: {Rh} -> {Rh_sym}")
print(f"O: {O} -> {O_sym}")

Position originale et transformée des atomes:
Cu: [0.0, 0.0, 0.0] -> [0. 0. 0.]
Rh: [0.5, 0.5, 0.5] -> [-0.5 -0.5 -0.5]
O: [0.25, 0.25, 0.75] -> [-0.75 -0.25 -0.25]


### Première symétrie sur l'atome de Cu

In [118]:
print("Opération de symétrie 1: \n")
print("L'atome de départ est Cu et a comme coordonnées: {}\n".format(Cu))
print("On lui applique une symétrie de plan miroir (z,y,x) :\n")
print(S1.rotation_matrix, "*", Cu, '+', S1.translation_vector)
print("L'atome d'arrivée a comme coordonnées : {}\n".format(Cu_sym))

# Visualisation avec Jupyter-JSmol
v1 = quick_view(structure)
display(v1)
v1.script("draw symop 8 {atomno=1}")

Opération de symétrie 1: 

L'atome de départ est Cu et a comme coordonnées: [0.0, 0.0, 0.0]

On lui applique une symétrie de plan miroir (z,y,x) :

[[ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]] * [0.0, 0.0, 0.0] + [0. 0. 0.]
L'atome d'arrivée a comme coordonnées : [0. 0. 0.]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

### Deuxième symétrie sur l'atome de Rh

In [119]:
print("Opération de symétrie 2: \n")
print("L'atome de départ est Rh et a comme coordonnées: {}\n".format(Rh))
print("On lui applique une roto-inversion (-x-y-z,x,y) :")
print(S2.rotation_matrix, "*", Rh, '+', S2.translation_vector)
print("L'atome d'arrivée a comme coordonnées : {}\n".format(Rh_sym))

# Visualisation avec Jupyter-JSmol
v2 = quick_view(structure)
display(v2)
v2.script("draw symop 4 {atomno=7}")

Opération de symétrie 2: 

L'atome de départ est Rh et a comme coordonnées: [0.5, 0.5, 0.5]

On lui applique une roto-inversion (-x-y-z,x,y) :
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]] * [0.5, 0.5, 0.5] + [0. 0. 0.]
L'atome d'arrivée a comme coordonnées : [-0.5 -0.5 -0.5]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

### Troisième symétrie sur l'atome de Ru

In [120]:
print("Opération de symétrie 3: \n")
print("L'atome de départ est le O et a comme coordonnées: {}\n".format(O))
print("On lui applique une symétrie d'axe 2 (y,x,-x-y-z) : ")
print(S3.rotation_matrix, "*", O, '+', S3.translation_vector)
print("L'atome d'arrivé a comme coordonnées : {}\n".format(O_sym))
v3 = quick_view(structure)
display(v3)
v3.script("draw symop 7 {atomno=3}")

Opération de symétrie 3: 

L'atome de départ est le O et a comme coordonnées: [0.25, 0.25, 0.75]

On lui applique une symétrie d'axe 2 (y,x,-x-y-z) : 
[[ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]] * [0.25, 0.25, 0.75] + [0. 0. 0.]
L'atome d'arrivé a comme coordonnées : [-0.75 -0.25 -0.25]



JsmolView(layout=Layout(align_self='stretch', height='400px'))